# Этап 1: Подготовка данных

#### Загрузка данных

In [153]:
# Импорт стандартных библиотек
import pandas as pd
import numpy as np

import os

# Для разделения данных и предобработки
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Для построения модели LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# Для генетического алгоритма
from deap import base, creator, tools, algorithms

# Для оценки модели
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline

# Дополнительные библиотеки
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

import joblib

In [155]:
data = pd.read_csv('data/sp500_historical_data.csv')

#### Преобразование столбца даты

In [158]:
data['Date'] = pd.to_datetime(data['Date'])

#### Удаление данных за выходные дни

In [161]:
data = data[data['Date'].dt.weekday < 5]

#### Определение полного диапазона дат

In [164]:
full_date_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='B')  # 'B' обозначает бизнес-день

#### Проверка наличия данных по каждому тикеру

In [167]:
data_pivot = data.pivot_table(index='Date', columns='Ticker', values='Adj Close')
missing_percent = data_pivot.isnull().mean() * 100
tickers_to_remove = missing_percent[missing_percent > 5].index.tolist()
clean_data = data[~data['Ticker'].isin(tickers_to_remove)]

#### Проверка на дупликаты

In [170]:
clean_data = clean_data.drop_duplicates()

#### Обработка подозрительно низкого объема торгов

In [173]:
volume_threshold = clean_data['Volume'].quantile(0.01)
low_volume_data = clean_data[clean_data['Volume'] <= volume_threshold]
low_volume_data.to_csv('data/low_volume_records.csv', index=False)

#### Проверка размерности данных

In [176]:
data_pivot_clean = clean_data.pivot_table(index='Date', columns='Ticker', values='Adj Close')
missing_dates_per_ticker = data_pivot_clean.isnull().sum()
tickers_with_missing_dates = missing_dates_per_ticker[missing_dates_per_ticker > 0].index.tolist()
clean_data = clean_data[~clean_data['Ticker'].isin(tickers_with_missing_dates)]

#### Сохранение очищенных данных

In [179]:
clean_tickers = clean_data['Ticker'].unique()
cleaned_data = data.sort_values(['Ticker', 'Date']).reset_index(drop=True)
clean_data.to_csv('data/cleaned_sp500_data.csv', index=False)

#### Разделение данных на обучающую и тестовую выборки

In [182]:
# Получение минимальной и максимальной даты
min_date = data['Date'].min()
max_date = data['Date'].max()

# Общий период времени
total_days = (max_date - min_date).days

# Дата разделения (80% от общего периода)
split_date = min_date + pd.Timedelta(days=int(total_days * 0.8))

print(f"Дата разделения данных: {split_date.date()}")
# Обучающая выборка: данные до даты разделения (включительно)
train_data = data[data['Date'] <= split_date]

# Тестовая выборка: данные после даты разделения
test_data = data[data['Date'] > split_date]
print(f"Размер обучающей выборки: {len(train_data)} записей")
print(f"Размер тестовой выборки: {len(test_data)} записей")


# Сохранение обучающей выборки
train_data.to_csv('data/sp500_train_data.csv', index=False)

# Сохранение тестовой выборки
test_data.to_csv('data/sp500_test_data.csv', index=False)

Дата разделения данных: 2022-05-25
Размер обучающей выборки: 298427 записей
Размер тестовой выборки: 74756 записей


#### Сбор информации о секторах

In [195]:
# Получение списка уникальных тикеров
tickers = data['Ticker'].unique()

# Инициализация словаря для хранения информации о секторах
ticker_sectors = {}

# Получение информации о секторах для каждого тикера
for ticker in tickers:
    try:
        stock_info = yf.Ticker(ticker).info
        sector = stock_info.get('sector', 'Unknown')
        ticker_sectors[ticker] = sector
    except Exception as e:
        print(f"Не удалось получить данные для {ticker}: {e}")
        ticker_sectors[ticker] = 'Unknown'

# Создание DataFrame из словаря
sectors_df = pd.DataFrame.from_dict(ticker_sectors, orient='index', columns=['Sector'])

sectors_df.to_csv('data/sectors.csv')

#### Сохранение волатильности

In [198]:
for ticker in tickers:
    # Получение данных для тикера
    df_ticker = data[data['Ticker'] == ticker].sort_values('Date')

    # Используем 'Adj Close' в качестве целевой переменной
    close_prices = df_ticker['Adj Close'].values.reshape(-1, 1)
    
    # Расчет доходностей для оценки волатильности
    returns = np.log(close_prices[1:] / close_prices[:-1])

    # Оценка волатильности (стандартное отклонение доходностей)
    volatility_value = np.std(returns)
    volatility[ticker] = volatility_value
    
# Преобразование словаря волатильности в DataFrame
volatility_df = pd.DataFrame.from_dict(volatility, orient='index', columns=['Volatility'])
volatility_df.to_csv('data/volatility.csv')

# Этап 2: Построение и оптимизация модели LSTM с помощью GA

#### Загрузка очищенных данных

In [189]:
# Загрузка обучающих данных
train_data = pd.read_csv('data/sp500_train_data.csv')

# Преобразование столбца 'Date' в формат datetime
train_data['Date'] = pd.to_datetime(train_data['Date'])

# Сортировка данных
train_data = train_data.sort_values(['Ticker', 'Date']).reset_index(drop=True)

#### Подготовка данных

In [56]:
# Определение параметров
TIME_STEPS = 30  # Длина последовательности для входа в LSTM

# Инициализация словарей для хранения результатов
models = {}
scalers = {}
volatility = {}
performance = {}

#### Установка random для воспроизведения

In [59]:
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

#### Определение функций для подготовки данных

In [62]:
def create_sequences(data, time_steps=TIME_STEPS):
    X = []
    y = []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

#### Определение функции оценки модели

In [65]:
import tensorflow.keras.backend as K

def evaluate_model(individual, X_train, y_train, X_val, y_val):
    # Unpack hyperparameters
    n_layers = individual[0]
    n_neurons = individual[1]
    dropout_rate = individual[2]
    learning_rate = individual[3]

    # Build the model
    K.clear_session()
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    if n_layers == 1:
        # Single LSTM layer
        model.add(LSTM(units=n_neurons, return_sequences=False))
        model.add(Dropout(dropout_rate))
    else:
        # First LSTM layer
        model.add(LSTM(units=n_neurons, return_sequences=True))
        model.add(Dropout(dropout_rate))
        # Middle LSTM layers
        for _ in range(n_layers - 2):
            model.add(LSTM(units=n_neurons, return_sequences=True))
            model.add(Dropout(dropout_rate))
        # Last LSTM layer
        model.add(LSTM(units=n_neurons, return_sequences=False))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Train the model
    history = model.fit(X_train, y_train, epochs=5, batch_size=32,
                        validation_data=(X_val, y_val), verbose=0)

    # Evaluate validation loss
    val_loss = history.history['val_loss'][-1]

    return val_loss,

#### Определение процесса генетического алгоритма

In [68]:
def custom_mutation(individual, indpb):
    for i in range(len(individual)):
        if random.random() < indpb:
            if i == 0:  # n_layers (целое число)
                individual[i] = random.randint(1, 2)  # Максимум 2 слоя
            elif i == 1:  # n_neurons (целое число)
                individual[i] = random.randint(32, 128)  # От 32 до 128 нейронов
            elif i == 2:  # dropout_rate (вещественное число)
                individual[i] = random.uniform(0.0, 0.3)  # От 0.0 до 0.3
            elif i == 3:  # learning_rate (вещественное число)
                individual[i] = random.uniform(0.0005, 0.005)  # Уменьшенный диапазон
    return individual,

In [70]:
def optimize_model_with_ga(X_train, y_train, X_val, y_val, n_generations=3, population_size=5):
    # Определение индивидуала и функции приспособленности
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Минимизируем валидационную ошибку
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    # Определение атрибутов для каждого гиперпараметра
    toolbox.register("attr_n_layers", random.randint, 1, 2)
    toolbox.register("attr_n_neurons", random.randint, 32, 128)
    toolbox.register("attr_dropout_rate", random.uniform, 0.0, 0.3)
    toolbox.register("attr_learning_rate", random.uniform, 0.0005, 0.005)

    # Создание индивидуала
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_n_layers, toolbox.attr_n_neurons, toolbox.attr_dropout_rate, toolbox.attr_learning_rate), n=1)

    # Создание популяции
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Определение функции оценки
    def eval_function(individual):
        return evaluate_model(individual, X_train, y_train, X_val, y_val)

    toolbox.register("evaluate", eval_function)
    toolbox.register("mate", tools.cxUniform, indpb=0.5)
    toolbox.register("mutate", custom_mutation, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Инициализация популяции
    pop = toolbox.population(n=population_size)

    # Использование Hall of Fame для отслеживания лучшего решения
    hof = tools.HallOfFame(1)

    # Параллельная оценка популяции
    def eval_population(population):
        fitnesses = Parallel(n_jobs=-1)(delayed(toolbox.evaluate)(ind) for ind in population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

    # Основной цикл GA
    for gen in range(n_generations):
        print(f"Поколение {gen+1}/{n_generations}")

        # Оценка популяции
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        if invalid_ind:
            eval_population(invalid_ind)

        # Обновление Hall of Fame
        hof.update(pop)

        # Применение операторов GA
        offspring = algorithms.varAnd(pop, toolbox, cxpb=0.5, mutpb=0.2)

        # Оценка потомков
        invalid_offspring = [ind for ind in offspring if not ind.fitness.valid]
        if invalid_offspring:
            eval_population(invalid_offspring)
        
        # Вывод лучшей валидационной ошибки в текущем поколении
        best = tools.selBest(pop, k=1)[0]
        print(f"Лучшая валидационная ошибка: {best.fitness.values[0]}")

        # Выбор новой популяции
        pop = toolbox.select(pop + offspring, k=len(pop))

    # Возвращаем лучший найденный индивидуал
    best_individual = hof[0]
    return best_individual

#### Обучение модели для каждого тикера

In [ ]:
# Создание каталогов для сохранения моделей и графиков
if not os.path.exists('models'):
    os.makedirs('models')
if not os.path.exists('plots'):
    os.makedirs('plots')

tickers = train_data['Ticker'].unique()

# Цикл по каждому тикеру
for ticker in tickers:
    if not os.path.isfile(f'models/{ticker}_model.keras'):
        print(f"Обработка тикера: {ticker}")

        # Получение данных для тикера
        df_ticker = train_data[train_data['Ticker'] == ticker].sort_values('Date')

        # Используем 'Adj Close' в качестве целевой переменной
        close_prices = df_ticker['Adj Close'].values.reshape(-1, 1)

        # Проверка достаточности данных
        if len(close_prices) <= TIME_STEPS:
            print(f"Недостаточно данных для тикера {ticker}")
            continue

        # Масштабирование данных
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)
        scalers[ticker] = scaler
        
        # Сохранение скейлера
        scaler_filename = f'scalers/{ticker}_scaler.save'
        joblib.dump(scaler, scaler_filename)

        # Создание последовательностей
        X, y = create_sequences(scaled_data)

        # Проверка достаточности последовательностей
        if len(X) == 0:
            print(f"Недостаточно последовательностей для тикера {ticker}")
            continue

        # Разделение данных на обучающую и тестовую выборки (80% на обучение)
        split_index = int(0.8 * len(X))
        X_train_full, X_test = X[:split_index], X[split_index:]
        y_train_full, y_test = y[:split_index], y[split_index:]

        # Дополнительное разделение обучающих данных на обучение и валидацию (80% на обучение)
        val_split_index = int(0.8 * len(X_train_full))
        X_train, X_val = X_train_full[:val_split_index], X_train_full[val_split_index:]
        y_train, y_val = y_train_full[:val_split_index], y_train_full[val_split_index:]

        # Оптимизация модели с помощью GA
        best_hyperparams = optimize_model_with_ga(X_train, y_train, X_val, y_val, n_generations=3, population_size=5)

        # Извлечение лучших гиперпараметров
        n_layers = best_hyperparams[0]
        n_neurons = best_hyperparams[1]
        dropout_rate = best_hyperparams[2]
        learning_rate = best_hyperparams[3]

        # Построение финальной модели с лучшими гиперпараметрами
        K.clear_session()
        model = Sequential()
        model.add(Input(shape=(X_train_full.shape[1], X_train_full.shape[2])))

        if n_layers == 1:
            # Single LSTM layer
            model.add(LSTM(units=n_neurons, return_sequences=False))
            model.add(Dropout(dropout_rate))
        else:
            # First LSTM layer
            model.add(LSTM(units=n_neurons, return_sequences=True))
            model.add(Dropout(dropout_rate))
            # Middle LSTM layers
            for _ in range(n_layers - 2):
                model.add(LSTM(units=n_neurons, return_sequences=True))
                model.add(Dropout(dropout_rate))
            # Last LSTM layer
            model.add(LSTM(units=n_neurons, return_sequences=False))
            model.add(Dropout(dropout_rate))

        model.add(Dense(1))

        # Compile the model
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        # Обучение финальной модели на полной обучающей выборке
        model.fit(X_train_full, y_train_full, epochs=20, batch_size=32, verbose=0)

        # Оценка модели на тестовых данных
        y_pred = model.predict(X_test)
        y_pred_inverse = scaler.inverse_transform(y_pred)
        y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Расчет метрик оценки
        mse = mean_squared_error(y_test_inverse, y_pred_inverse)
        mae = mean_absolute_error(y_test_inverse, y_pred_inverse)

        performance[ticker] = {'MSE': mse, 'MAE': mae}

        # Сохранение модели
        model.save(f'models/{ticker}_model.keras')
        models[ticker] = model

        # Визуализация результатов
        plt.figure(figsize=(10, 6))
        plt.plot(y_test_inverse, label='Реальная цена')
        plt.plot(y_pred_inverse, label='Предсказанная цена')
        plt.title(f'Предсказание цены для {ticker}')
        plt.xlabel('Время')
        plt.ylabel('Цена')
        plt.legend()
        plt.savefig(f'plots/{ticker}_prediction.png')
        plt.close()
    else:
        print(f"Модель для тикера: {ticker} уже создана")

#### Сохранение метрик модели

In [ ]:
# Преобразование словаря производительности в DataFrame
performance_df = pd.DataFrame.from_dict(performance, orient='index')
performance_df.to_csv('data/model_performance.csv')

# Этап 3: Формирование портфеля

#### Загрузка сохраненных данных

In [201]:
# Загрузка тестовых данных
test_data = pd.read_csv('data/sp500_test_data.csv')
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data = test_data.sort_values(['Ticker', 'Date']).reset_index(drop=True)

# Определяем уникальные даты
unique_dates = test_data['Date'].unique()
total_dates = len(unique_dates)
train_size = int(total_dates * 0.8)
train_dates = unique_dates[:train_size]
test_dates = unique_dates[train_size:]

# Получение списка чистых тикеров
clean_tickers = test_data['Ticker'].unique()

# Загрузка волатильности из файла
volatility_df = pd.read_csv('data/volatility.csv', index_col=0)
volatility = volatility_df['Volatility'].to_dict()

# Загрузка информации о секторах
sectors_df = pd.read_csv('data/sectors.csv', index_col=0)

#### Загрузка сохраненных моделей

In [ ]:
from tensorflow.keras.models import load_model

# Загрузка сохраненных моделей
models = {}
scalers = {}
for ticker in clean_tickers:
    scaler_filename = f'scalers/{ticker}_scaler.save'
    if os.path.exists(scaler_filename):
        scalers[ticker] = joblib.load(scaler_filename)
    else:
        print(f"Скейлер для тикера {ticker} не найден.")
        continue
    model_path = f'models/{ticker}_model.keras'
    if os.path.exists(model_path):
        models[ticker] = load_model(model_path)
    else:
        print(f"Модель для тикера {ticker} не найдена.")

#### Подготовка данных для предсказания

In [108]:
TIME_STEPS = 30  # Длина последовательности для входа в LSTM

# Дата, на которую мы делаем предсказание (последняя доступная дата)
prediction_date = data['Date'].max()

#### Составление портфеля

In [ ]:
# Словарь для хранения ожидаемой доходности и волатильности
expected_returns = {}

for ticker in clean_tickers:
    if ticker not in models:
        continue
    # Получаем данные для тикера
    df_ticker = data[data['Ticker'] == ticker].sort_values('Date')
    
    # Проверяем, есть ли достаточное количество данных
    if len(df_ticker) < TIME_STEPS + 7:
        continue
    
    # Получаем последние TIME_STEPS данных для предсказания
    last_data = df_ticker.iloc[-TIME_STEPS:]
    last_close_prices = last_data['Adj Close'].values.reshape(-1, 1)
    
    # Масштабируем данные
    scaler = scalers[ticker]
    scaled_data = scaler.transform(last_close_prices)
    
    # Формируем входные данные для модели
    X_input = np.array([scaled_data])
    
    # Делаем предсказание на 7 дней вперед
    # Рекурсивное предсказание
    predictions = []
    current_input = X_input.copy()
    for _ in range(7):
        predicted_price = models[ticker].predict(current_input)
        predictions.append(predicted_price[0, 0])
        
        # Преобразуем predicted_price в правильную форму
        predicted_price_reshaped = predicted_price.reshape((1, 1, 1))
        
        # Обновляем current_input, удаляя первый временной шаг и добавляя предсказанное значение
        current_input = np.concatenate((current_input[:, 1:, :], predicted_price_reshaped), axis=1)
    
    # Обратное масштабирование предсказанных цен
    predicted_prices = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    
    # Текущая цена
    current_price = last_close_prices[-1, 0]
    
    # Предсказанная цена через 7 дней
    predicted_price_7d = predicted_prices[-1, 0]
    
    # Рассчитываем ожидаемую доходность
    expected_return = (predicted_price_7d - current_price) / current_price
    
    # Сохраняем ожидаемую доходность и волатильность
    expected_returns[ticker] = {
        'Expected Return': expected_return,
        'Volatility': volatility.get(ticker, np.nan),
        'Sector': sectors_df[sectors_df['Ticker'] == ticker]['Sector'].values[0] if ticker in sectors_df['Ticker'].values else 'Unknown'
    }

#### Сортируем акции по коэффициенту Шарпа и отбираем топ-10 с учетом диверсификации

In [124]:
# Создаем DataFrame с информацией о тикерах
expected_returns_df = pd.DataFrame.from_dict(expected_returns, orient='index')

# Удаляем строки с отсутствующими значениями
expected_returns_df = expected_returns_df.dropna()

# Рассчитываем коэффициент Шарпа
expected_returns_df['Sharpe Ratio'] = expected_returns_df['Expected Return'] / expected_returns_df['Volatility']

In [126]:
# Сортируем акции по коэффициенту Шарпа в порядке убывания
expected_returns_df = expected_returns_df.sort_values(by='Sharpe Ratio', ascending=False)

# Отбираем топ-10 акций с учетом диверсификации по секторам
selected_tickers = []
sector_counts = {}

for idx, row in expected_returns_df.iterrows():
    sector = row['Sector']
    # Инициализируем счетчик для сектора
    if sector not in sector_counts:
        sector_counts[sector] = 0
    # Проверяем, сколько акций из этого сектора уже выбрано
    if sector_counts[sector] < 2:  # Ограничение: не более 2 акций из одного сектора
        selected_tickers.append(idx)
        sector_counts[sector] += 1
    if len(selected_tickers) == 10:
        break

# Проверяем, что набрано 10 акций
if len(selected_tickers) < 10:
    # Если не удалось набрать 10 акций с учетом ограничений по секторам, добираем оставшиеся акции
    remaining_tickers = expected_returns_df.index.difference(selected_tickers)
    for ticker in remaining_tickers:
        selected_tickers.append(ticker)
        if len(selected_tickers) == 10:
            break

# Финальный DataFrame с выбранными акциями
portfolio_df = expected_returns_df.loc[selected_tickers]

print("Выбранные акции для портфеля:")
print(portfolio_df[['Expected Return', 'Volatility', 'Sharpe Ratio', 'Sector']])

Выбранные акции для портфеля:
      Expected Return  Volatility  Sharpe Ratio                  Sector
NCLH         0.244325    0.019892     12.282724       Consumer Cyclical
UAL          0.212057    0.019892     10.660564             Industrials
MRO          0.188018    0.019892      9.452060                  Energy
BAX          0.179797    0.019892      9.038778              Healthcare
SWK          0.176865    0.019892      8.891355             Industrials
MTCH         0.146464    0.019892      7.363024  Communication Services
PARA         0.146294    0.019892      7.354493  Communication Services
KMX          0.139848    0.019892      7.030437       Consumer Cyclical
CPAY         0.123951    0.019892      6.231249              Technology
PRU          0.114168    0.019892      5.739436      Financial Services


#### Расчет долей инвестиций по критерию Келли

In [129]:
# Расчет долей инвестиций по критерию Келли
portfolio_df['Kelly Fraction'] = portfolio_df.apply(
    lambda row: row['Expected Return'] / (row['Volatility'] ** 2), axis=1
)

# Убираем отрицательные значения и заменяем их на 0
portfolio_df['Kelly Fraction'] = portfolio_df['Kelly Fraction'].apply(lambda x: max(x, 0))

# Ограничиваем максимальную долю в одной акции (например, 20%)
max_fraction = 0.2
portfolio_df['Kelly Fraction'] = portfolio_df['Kelly Fraction'].apply(lambda x: min(x, max_fraction))

# Проверяем сумму долей
total_fraction = portfolio_df['Kelly Fraction'].sum()

# Если сумма долей больше 1, нормализуем доли
if total_fraction > 1:
    portfolio_df['Kelly Fraction'] = portfolio_df['Kelly Fraction'] / total_fraction

# Финальные доли инвестиций
portfolio_df['Investment Fraction'] = portfolio_df['Kelly Fraction']

print("Распределение капитала по акциям:")
print(portfolio_df[['Expected Return', 'Volatility', 'Sharpe Ratio', 'Sector', 'Investment Fraction']])

Распределение капитала по акциям:
      Expected Return  Volatility  Sharpe Ratio                  Sector  \
NCLH         0.244325    0.019892     12.282724       Consumer Cyclical   
UAL          0.212057    0.019892     10.660564             Industrials   
MRO          0.188018    0.019892      9.452060                  Energy   
BAX          0.179797    0.019892      9.038778              Healthcare   
SWK          0.176865    0.019892      8.891355             Industrials   
MTCH         0.146464    0.019892      7.363024  Communication Services   
PARA         0.146294    0.019892      7.354493  Communication Services   
KMX          0.139848    0.019892      7.030437       Consumer Cyclical   
CPAY         0.123951    0.019892      6.231249              Technology   
PRU          0.114168    0.019892      5.739436      Financial Services   

      Investment Fraction  
NCLH                  0.1  
UAL                   0.1  
MRO                   0.1  
BAX                   0.1  


#### Оценка доходности портфеля через неделю

In [132]:
# Рассчитываем фактическую доходность каждой акции через неделю
actual_returns = {}
for ticker in selected_tickers:
    df_ticker = data[data['Ticker'] == ticker].sort_values('Date').reset_index(drop=True)
    idx_current = df_ticker[df_ticker['Date'] == prediction_date].index
    if len(idx_current) == 0:
        continue
    idx_current = idx_current[0]
    if idx_current + 7 >= len(df_ticker):
        continue
    actual_price_7d = df_ticker.iloc[idx_current + 7]['Adj Close']
    current_price = df_ticker.iloc[idx_current]['Adj Close']
    actual_return = (actual_price_7d - current_price) / current_price
    actual_returns[ticker] = actual_return

# Добавляем фактическую доходность в DataFrame портфеля
portfolio_df['Actual Return'] = portfolio_df.index.map(actual_returns)

# Заполняем отсутствующие значения нулями
portfolio_df['Actual Return'] = portfolio_df['Actual Return'].fillna(0)

# Рассчитываем общую доходность портфеля
portfolio_return = (portfolio_df['Actual Return'] * portfolio_df['Investment Fraction']).sum()

print(f"Фактическая доходность портфеля через неделю: {portfolio_return * 100:.2f}%")

Фактическая доходность портфеля через неделю: 0.00%


# Этап 4: пересмотр портфеля

#### Загрузка данных по индексу S&P 500

In [210]:
# Загрузка данных по индексу
sp500_data = pd.read_csv('data/sp500_index_data.csv')

# Преобразование столбца 'Date' в формат datetime
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])

# Сортировка данных по дате
sp500_data = sp500_data.sort_values('Date').reset_index(drop=True)

#### Расчет ежедневной доходности индекса

In [213]:
# Расчет ежедневной доходности индекса
sp500_data['Return'] = sp500_data['Adj Close'].pct_change()

#### Сравнение доходности портфелей

In [218]:
# Шаг 1: Загрузка тестовых данных
test_data = pd.read_csv('data/sp500_test_data.csv')
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data = test_data.sort_values(['Date']).reset_index(drop=True)

# Шаг 2: Получение списка уникальных торговых дат
test_dates = sorted(test_data['Date'].unique())

# Шаг 3: Выбор каждой 5-й даты
review_dates = test_dates[::5]

# Шаг 4: Преобразование в DatetimeIndex, если необходимо
review_dates = pd.to_datetime(review_dates)

# Теперь review_dates содержит даты пересмотра портфеля
print("Даты пересмотра портфеля:")
print(review_dates)

Даты пересмотра портфеля:
DatetimeIndex(['2022-05-26', '2022-06-03', '2022-06-10', '2022-06-17',
               '2022-06-27', '2022-07-05', '2022-07-12', '2022-07-19',
               '2022-07-26', '2022-08-02', '2022-08-09', '2022-08-16',
               '2022-08-23', '2022-08-30', '2022-09-07', '2022-09-14',
               '2022-09-21', '2022-09-28', '2022-10-05', '2022-10-12',
               '2022-10-19', '2022-10-26', '2022-11-02', '2022-11-09',
               '2022-11-16', '2022-11-23', '2022-12-01', '2022-12-08',
               '2022-12-15', '2022-12-22', '2022-12-30'],
              dtype='datetime64[ns]', freq=None)


#### Пересмотр портфеля

In [221]:
portfolio_history = []
portfolio_returns = []
index_returns = []
dates = []

for i in range(len(review_dates) - 1):
    current_date = review_dates[i]
    next_date = review_dates[i + 1]
    print(f"Пересмотр портфеля на дату: {current_date.date()}")

    # Словарь для хранения ожидаемой доходности и волатильности
    expected_returns = {}

    for ticker in clean_tickers:
        if ticker not in models:
            continue
        # Получаем данные для тикера до текущей даты
        df_ticker = test_data[(test_data['Ticker'] == ticker) & (test_data['Date'] <= current_date)].sort_values('Date')

        # Проверяем, есть ли достаточное количество данных
        if len(df_ticker) < TIME_STEPS + 7:
            continue

        # Получаем последние TIME_STEPS данных для предсказания
        last_data = df_ticker.iloc[-TIME_STEPS:]
        last_close_prices = last_data['Adj Close'].values.reshape(-1, 1)

        # Масштабируем данные
        scaler = scalers[ticker]
        scaled_data = scaler.transform(last_close_prices)

        # Формируем входные данные для модели
        X_input = np.array([scaled_data])

        # Делаем предсказание на 7 дней вперед
        # Рекурсивное предсказание
        predictions = []
        current_input = X_input.copy()
        for _ in range(7):
            predicted_price = models[ticker].predict(current_input)
            predictions.append(predicted_price[0, 0])

            # Преобразуем predicted_price в правильную форму
            predicted_price_reshaped = predicted_price.reshape((1, 1, 1))

            # Обновляем current_input
            current_input = np.concatenate((current_input[:, 1:, :], predicted_price_reshaped), axis=1)

        # Обратное масштабирование предсказанных цен
        predicted_prices = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

        # Текущая цена
        current_price = last_close_prices[-1, 0]

        # Предсказанная цена через 7 дней
        predicted_price_7d = predicted_prices[-1, 0]

        # Рассчитываем ожидаемую доходность
        expected_return = (predicted_price_7d - current_price) / current_price

        # Сохраняем ожидаемую доходность и волатильность
        expected_returns[ticker] = {
            'Expected Return': expected_return,
            'Volatility': volatility.get(ticker, np.nan),
            'Sector': sectors_df[sectors_df['Ticker'] == ticker]['Sector'].values[0] if ticker in sectors_df['Ticker'].values else 'Unknown'
        }

    # Преобразуем словарь в DataFrame
    expected_returns_df = pd.DataFrame.from_dict(expected_returns, orient='index').dropna()

    # Продолжаем только если есть данные
    if expected_returns_df.empty:
        print("Нет доступных данных для составления портфеля на эту дату.")
        continue

    # Остальная часть кода остается без изменений...
    # Отбор акций, расчет долей по критерию Келли, оценка доходности и т.д.

Пересмотр портфеля на дату: 2022-05-26
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-06-03
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-06-10
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-06-17
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-06-27
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-07-05
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-07-12
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-07-19
Нет доступных данных для составления портфеля на эту дату.
Пересмотр портфеля на дату: 2022-07-26
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

KeyError: 'Ticker'